<a href="https://colab.research.google.com/github/KhaledRobot/ML/blob/main/Copy_of_Court1_agent_ex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: read)

In [ ]:


# Two–Lawyer Agents (Defense & Prosecution)


from __future__ import annotations
import os
from typing import List, Dict
from huggingface_hub import InferenceClient

class LawyerAgent:


    def __init__(self,
                 name: str,
                 system_prompt: str,
                 model: str = "microsoft/Phi-3-mini-4k-instruct",
                 length_instruction: str = "Please keep your responses between 100 and 200 words."):
        self.name = name
        self.system_prompt = system_prompt.strip()
        self.length_instruction = length_instruction
        self.history: List[Dict[str, str]] = []      # list of {"role": ..., "content": ...}
        self.client = InferenceClient(
            model,
            token=os.getenv("HF_API_TOKEN")           # make sure this env‑var is set
        )

    # ---- helper for HF prompt formatting ----------
    def _format_prompt(self, user_msg: str) -> str:
        """
        Formats a full prompt that includes
        * system prompt
        * prior turns
        * new user message
        """
        messages = [{"role": "system", "content": self.system_prompt}]
        messages.extend(self.history)
        messages.append({"role": "user", "content": user_msg})

        # HF text-generation endpoints expect a single string.

        prompt = ""
        for m in messages:
            prompt += f"<|{m['role']}|>\n{m['content']}\n"
        prompt += "<|assistant|>\n"
        return prompt

    # ---- produce a reply --------------------------
    def respond(self, user_msg: str, **gen_kwargs) -> str:
        full_msg = f"{user_msg}\n\n{self.length_instruction}"
        prompt = self._format_prompt(user_msg)

        # Ensure max_new_tokens is set if not passed
        gen_kwargs.setdefault("max_new_tokens", 256)

        completion = self.client.text_generation(
            prompt,
            temperature=0.7,
            do_sample=True,
            stream=False,
            **gen_kwargs
    )

        answer = completion.strip()
        self.history.append({"role": "user", "content": user_msg})
        self.history.append({"role": "assistant", "content": answer})
        return answer



# System prompts

DEFENSE_SYSTEM = """
You are **Alex Carter**, lead *defense counsel*.
Goals:
• Protect the constitutional rights of the defendant.
• Raise reasonable doubt by pointing out missing evidence or alternative explanations.
• Be respectful to the Court and to opposing counsel.
Style:
• Crisp, persuasive, grounded in precedent and facts provided.
• When citing precedent: give short case name + year (e.g., *Miranda v. Arizona* (1966)).
Ethics:
• Do not fabricate evidence; admit uncertainty when required.
"""

PROSECUTION_SYSTEM = """
You are **Jordan Blake**, *Assistant District Attorney* for the State.
Goals:
• Present the strongest good‑faith case against the accused.
• Lay out facts logically, citing exhibits or witness statements when available.
• Anticipate and rebut common defense arguments.
Style:
• Formal but plain English; persuasive, with confident tone.
Ethics:
• Duty is to justice, not merely to win. Concede points when ethically required.
"""

PLAINTIFF_SYSTEM = """
You are **Garcia**, Plaintiff in this case.
Goals:
• Clearly articulate the defendant’s wrongful act or omission.
• Demonstrate causal link between the defendant’s actions and your damages.
• Request specific, legally viable remedies (e.g., compensation, injunction).
Style:
• Fact-driven; avoid emotional appeals.
• Use bullet points or numbered lists for clarity (e.g., "First, the defendant failed to…").
• Cite evidence if available (e.g., "Per Exhibit B, the contract required…").
Ethics:
• Do not exaggerate claims; stick to provable facts.
• Acknowledge counterarguments if objectively valid.
"""

WITNESS_SYSTEM = """
Role: You are an eyewitness under oath.
Rules:
• ONLY testify to what you directly perceived (sight, sound, etc.).
• If asked for inferences, respond: "I can only state what I observed."
• Use clear timestamps/locations when possible (e.g., "At 10:15 AM, near the intersection…").
Style:
• Short, declarative sentences.
• Default to "I don’t recall" for uncertain details.
Ethics:
• Never speculate or adopt others’ accounts as your own.
• Correct yourself if contradicted by evidence.
"""

DEFENDANT_SYSTEM = """
Role: You are the defendant.
Goals:
• Rebut accusations with factual explanations or alibis.
• Highlight inconsistencies in the plaintiff/prosecution’s case.
• If guilty, focus on mitigating factors (e.g., intent, context).
Style:
• Respectful but assertive (e.g., "I disagree because…").
• Use "I" statements for personal knowledge ("I was at work until 6PM").
Ethics:
• Do not lie or invent details.
• Admit fault if evidence is irrefutable.
"""

JUDGE_SYSTEM = """
Role: You are the presiding Judge.
Duties:
• Summarize key arguments from both sides (e.g., "Defense contends… Prosecution argues…").
• Apply relevant legal standards (e.g., "Under Daubert, this evidence is inadmissible because…").
• Render verdicts with succinct reasoning (1-3 sentences).
Style:
• Neutral, authoritative tone.
• Reference exhibits/witness credibility when decisive.
Ethics:
• Prioritize procedural fairness over personal bias.
• Explain rulings to ensure parties understand outcomes.
"""

# the two agents

defense = LawyerAgent("Defense", DEFENSE_SYSTEM)
prosecution = LawyerAgent("Prosecution", PROSECUTION_SYSTEM)

# New agents
plaintiff = LawyerAgent("Plaintiff", PLAINTIFF_SYSTEM)
witness = LawyerAgent("Witness", WITNESS_SYSTEM)
defendant = LawyerAgent("Defendant", DEFENDANT_SYSTEM)
judge = LawyerAgent("Judge", JUDGE_SYSTEM)

class TrialManager:
    def __init__(self, agents: dict, case_background: str):
        self.agents = agents
        self.case = case_background
        self.transcript = []

    def record(self, speaker, message):
        self.transcript.append(f"{speaker.upper()}: {message}\n")

    def run_opening_statements(self):
        print("==== Opening Statements ====\n")
        prosecution_msg = self.agents["prosecution"].respond(f"Opening statement to the Court. Background: {self.case}")
        defense_msg = self.agents["defense"].respond("Opening statement to the Court responding to the prosecution.")
        print("PROSECUTION:", prosecution_msg, "\n")
        print("DEFENSE   :", defense_msg, "\n")
        self.record("prosecution", prosecution_msg)
        self.record("defense", defense_msg)

    def run_interrogation(self):
        print("==== Interrogation Phase ====\n")
        prose_q = self.agents["prosecution"].respond("Interrogate the witness about what they saw.")
        witness_a1 = self.agents["witness"].respond(prose_q)
        prose_follow = self.agents["prosecution"].respond(f"The witness said: {witness_a1}. Respond or continue questioning.")

        defense_q = self.agents["defense"].respond("Cross-examine the witness based on their testimony.")
        witness_a2 = self.agents["witness"].respond(defense_q)
        defense_follow = self.agents["defense"].respond(f"The witness said: {witness_a2}. Respond or continue questioning.")
        print("PROSECUTION Q:", prose_q)
        print("WITNESS A1   :", witness_a1)
        print("PROSECUTION FOLLOW-UP:", prose_follow)
        print("DEFENSE Q    :", defense_q)
        print("WITNESS A2   :", witness_a2)
        print("DEFENSE FOLLOW-UP:", defense_follow)


        for role, msg in [("prosecution", prose_q), ("witness", witness_a1),
                          ("prosecution", prose_follow), ("defense", defense_q),
                          ("witness", witness_a2), ("defense", defense_follow)]:
            self.record(role, msg)

    def run_closing_statements(self):
        print("==== Closing Statements ====\n")
        prose = self.agents["prosecution"].respond("Give your closing statement.")
        defense = self.agents["defense"].respond("Give your closing statement.")
        print("PROSECUTION:", prose, "\n")
        print("DEFENSE   :", defense, "\n")
        self.record("prosecution", prose)
        self.record("defense", defense)

    def judge_verdict(self):
        print("==== Judge's Verdict ====\n")
        max_chars = 256  # tweak this if needed
        summary = "\n".join(self.transcript)[-max_chars:]
        summary = summary.strip().split("####")[0].strip()


        verdict = self.agents["judge"].respond(
            f"You are the judge. Here's the trial transcript:\n{summary}\nNow give your verdict and explain your reasoning.",
            max_new_tokens=300
        )

        print("JUDGE:", verdict)
        self.record("judge", verdict)



# ==== RUN THE SIMULATION ====

case_background = (
    "The State alleges that John Doe stole proprietary algorithms from his former employer "
    "and used them at a competitor. The charge is felony theft of trade secrets. "
    "No physical evidence shows direct copying, but server logs indicate large downloads "
    "two days before Doe resigned."
)

agents = {
    "prosecution": prosecution,
    "defense": defense,
    "plaintiff": plaintiff,
    "witness": witness,
    "defendant": defendant,
    "judge": judge,
}

trial = TrialManager(agents, case_background)

trial.run_opening_statements()
trial.run_interrogation()
trial.run_closing_statements()
trial.judge_verdict()


==== Opening Statements ====

PROSECUTION: Ladies and gentlemen of the jury,

Today, you will be asked to weigh the evidence presented by the State and determine the guilt of the defendant, John Doe. We are faced with a case that cuts to the core of what it means to protect the lifeblood of our economy—our intellectual property.

The defendant stands accused of the felony theft of trade secrets, a charge that carries with it profound implications for the integrity of our business community and the innovation that drives our society forward.

Our evidence is damning. While direct physical evidence of theft may not be present, the digital breadcrumbs lead us to a clear conclusion. Server logs show that shortly before his resignation, Mr. Doe had engaged in large downloads of proprietary algorithms from his former employer. These downloads were not merely routine or accidental, but deliberate and calculated, as the timing and the volume suggest.

The defense may argue that these downloads

In [ ]:
# ... (keep all your existing imports and LawyerAgent class)

class CourtAgent(LawyerAgent):
    def __init__(self, name: str, role: str, case_text: str):
        system_prompt = f"""
        **ROLE**: {role} in this case
        **CASE TEXT**: "{case_text[:300]}..."
        **RULES**:
        1. Respond ONLY about this specific case
        2. Never mention other cases/laws
        3. Keep responses under 100 words
        4. If unsure, say "Objection - beyond case scope"
        """
        super().__init__(name, system_prompt)
        self.case_text = case_text

# Load cases
import pandas as pd
from typing import List, Dict
import csv
cases_df = pd.read_csv("data.csv")

# Pick specific 10 indices (e.g., random or chosen manually to avoid first 10 always)
selected_indices = [1, 5, 10, 25, 30, 45, 100, 150, 200, 250]
cases_subset = cases_df.iloc[selected_indices]

# Process cases
results = []
MAX_CASES = min(10, len(cases_df))  # Handle case if fewer than 10 cases exist

for i, row in cases_df.head(MAX_CASES).iterrows():
        case_text = row.iloc[0]

        # Initialize agents (using your existing LawyerAgent class)
        prosecutor = LawyerAgent("Prosecutor",
                               f"You are prosecuting this case. Case details: {case_text[:300]}...")
        defense = LawyerAgent("Defense",
                             f"You are defending this case. Case details: {case_text[:300]}...")

        # Generate opening statements
        prosecution_stmt = prosecutor.respond("Present your opening arguments to the court")
        defense_stmt = defense.respond("Present your response to the prosecution's arguments")

        # Interrogation: Prosecution questions a witness
        interrogation_q = prosecutor.respond("Ask the witness what they saw")
        interrogation_a = defense.respond(f"The witness answered the prosecution: '{interrogation_q}'. Cross-examine the witness")

        judge = CourtAgent("Judge", "Judge", case_text)
        verdict = judge.respond(f"Prosecutor: {prosecution_stmt} | Defense: {defense_stmt}")

        # Save results
        results.append({
            'case_id': i,
            'case_preview': str(case_text)[:100] + "...",
            'prosecution': prosecution_stmt,
            'defense': defense_stmt,
            'interrogation_question': interrogation_q,
            'interrogation_answer': interrogation_a,
            'verdict': verdict
         })


In [ ]:
# Convert to DataFrame and save
results_df = pd.DataFrame(results)
results_df.to_csv('court_results.csv', index=False)

# Attempt download if in Colab
try:
    from google.colab import files
    files.download('court_results.csv')
except Exception as e:
    print("Download not available outside Colab. Error:", e)

# Preview results
print("\nSample outputs:")
for r in results[:2]:
    print(f"\nCase {r['case_id']}: {r['case_preview']}")
    print(f"Prosecution: {r['prosecution'][:100]}...")
    print(f"Defense: {r['defense'][:100]}...")

print(f"\n🎉 Saved {len(results_df)} case results to court_results.csv")
print("Columns:", results_df.columns.tolist())


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


Sample outputs:

Case 0:  S. THAKUR, J. Leave granted. These appeals are directed against an order dated 9th March, 2007 pass...
Prosecution: Your Honour,

On behalf of the appellants, we thank the Court for granting the leave to appeal again...
Defense: Your Honor, ladies and gentlemen of the court, the prosecution's arguments, while presented with con...

Case 1:  Markandey Katju, J. Leave granted. Heard learned companynsel for the appellant. None has appeared f...
Prosecution: Your Honor,


I appreciate the opportunity to present the opening arguments on behalf of the appella...
Defense: While I don't have specific details on this case, generally speaking, when responding to the prosecu...

🎉 Saved 10 case results to court_results.csv
Columns: ['case_id', 'case_preview', 'prosecution', 'defense', 'interrogation_question', 'interrogation_answer', 'verdict']
